In [1]:
"""
Fine-grained classification practice with Flower-17
"""

# Python Packages
import argparse
import os
import time
# 3rd Party Packages
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from sigopt import Connection
import pycm
# Initialize the optimizer and model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import save_model
# User Packages
from start.preprocessing import ImageToTensorPreprocessor, ResizePreprocessor, ColorSpacePreprocessor
from start.loader import ImageCachedDataset
from start.model import MobileNetV2
from keys.sigopt import token

In [2]:
WIDTH = 224
HEIGHT = WIDTH
CHANNELS = 1
BATCH_SIZE = 32
# Initialize optimizer
N_EPOCHS = 100
learning_rate = 0.01
decay_rate = learning_rate / N_EPOCHS
N_TRAINABLE_LAYERS = 152
timestamp = time.time()
OUTPUT_DIR = r'/home/share/education/deep_learning/pyimagesearch/models/flower-17-sigopt/'
EXPERIMENT_ID = 57907
nLoops = 1

In [3]:
# Load Flowers-17 dataset
dataset = ImageCachedDataset(
    preprocessors=[
        ResizePreprocessor(width=WIDTH, height=HEIGHT, aspect_preserving=True),
        ColorSpacePreprocessor(conversion=cv2.COLOR_BGR2GRAY),
        ImageToTensorPreprocessor()
    ],
    dataset_path=r'/home/share/dataset/flowers17'
)
(data, labels) = dataset.load(
    verbosity=80
)

print('data shape: {}'.format(data.shape))
print('labels shape: {}'.format(labels.shape))

classes = set(labels)

# Normalize data
data = data.astype(np.float) / 255.0

[INFO] Processing label: dandelion
[INFO] processed 80 dandelion images
[INFO] Processing label: coltsfoot
[INFO] processed 80 coltsfoot images
[INFO] Processing label: lilyvalley
[INFO] processed 80 lilyvalley images
[INFO] Processing label: cowslip
[INFO] processed 80 cowslip images
[INFO] Processing label: iris
[INFO] processed 80 iris images
[INFO] Processing label: snowdrop
[INFO] processed 80 snowdrop images
[INFO] Processing label: sunflower


[INFO] processed 80 sunflower images
[INFO] Processing label: tigerlily
[INFO] processed 80 tigerlily images
[INFO] Processing label: daisy
[INFO] processed 80 daisy images
[INFO] Processing label: pansy
[INFO] processed 80 pansy images
[INFO] Processing label: crocus
[INFO] processed 80 crocus images
[INFO] Processing label: daffodil
[INFO] processed 80 daffodil images
[INFO] Processing label: fritillary
[INFO] processed 80 fritillary images
[INFO] Processing label: buttercup


[INFO] processed 80 buttercup images
[INFO] Processing label: windflower
[INFO] processed 80 windflower images
[INFO] Processing label: files.txt
[INFO] Processing label: tulip
[INFO] processed 80 tulip images
[INFO] Processing label: bluebell
[INFO] processed 80 bluebell images
data shape: (1360, 224, 224, 3)
labels shape: (1360,)


In [4]:
# Setup data splits
# Partition into train and test splits
(trainX, testX, trainY, testY) = train_test_split(
    data, labels,
    test_size=0.2,
    random_state=int(time.time()),
    stratify=list(labels)
)
(valX, testX, valY, testY) = train_test_split(
    testX, testY,
    test_size=0.4,
    random_state=int(time.time()),
    stratify=list(testY)
)

# Binarize output to one hot vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
valY = lb.fit_transform(valY)
testY = lb.fit_transform(testY)

# Data augmentation
augmenter = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
# Sigopt
sigopt_connection = Connection(client_token=token)
"""
sigopt_experiment = sigopt_connection.experiments().create(
    name='Flowers-17-MobileNetV2-{}'.format(timestamp),
    parameters=[
        dict(name='dense_units', type='int', bounds=dict(min=128, max=512)),
        dict(name='dropout_rate', type='double', bounds=dict(min=0.2, max=0.6)),
        dict(name='regularization_strength', type='double', bounds=dict(min=0.0001, max=0.01))
    ],
    metrics=[dict(name='function_value')],
    parallel_bandwidth=1,
    observation_budget=30
)
"""
sigopt_experiment = sigopt_connection.experiments(EXPERIMENT_ID).fetch()
print("Created experiment: https://app.sigopt.com/experiment/" + sigopt_experiment.id)



Created experiment: https://app.sigopt.com/experiment/57907


In [6]:
# Run the optimization loop until observation budget is exhausted or local budget exhausted
while sigopt_experiment.progress.observation_count < sigopt_experiment.observation_budget or nLoops > 0:
    # Get SigOpt suggestions
    try:
        suggestion = sigopt_connection.experiments(sigopt_experiment.id).suggestions().create()
    except:
        pass

    # Build model 
    model = MobileNetV2.build({
        'width':        WIDTH,
        'height':       HEIGHT,
        'channels':     CHANNELS,
        'classes':      classes,
        'weights':      'imagenet',
        'dense_units':  242,
        'dropout_rate': 0.43,
        'regularization_strength': 0.0001,
    })
    
    """
    tb_callback = TensorBoard(
        log_dir='./logs/{}'.format(timestamp), 
        histogram_freq=2, 
        batch_size=BATCH_SIZE, 
        write_graph=False, 
        write_grads=False, 
        write_images=False, 
        embeddings_freq=0,
        embeddings_layer_names=None, 
        embeddings_metadata=None, 
        embeddings_data=None
    )
    """
    
    opt = SGD(
        lr=learning_rate,
        momentum=0,
        decay=0,
        nesterov=False
    )
    """
    opt = Adam(
        lr=learning_rate,
        beta_1=0.99,
        beta_2=0.999,
        epsilon=0.1,
        decay=decay_rate,
        amsgrad=False
    )
    """
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
    )
    
    # Train the network
    print('[INFO] training network...')
    history = model.fit_generator(
        augmenter.flow(trainX, trainY, 
                       batch_size=BATCH_SIZE),
        validation_data=(valX, valY),
        steps_per_epoch=len(trainX) // BATCH_SIZE,
        epochs=N_EPOCHS,
        callbacks=[],
        verbose=1
    )

    # Evaluate the network
    print('[INFO] evaluating network...')
    predictions = model.predict(testX, batch_size=BATCH_SIZE)
    
    cm = pycm.ConfusionMatrix(
        actual_vector=lb.inverse_transform(testY),
        predict_vector=lb.inverse_transform(predictions)
    )
    
    model_string = ('{0}_flowers-17-bw-valacc{1:.3f}-valloss{2:.3f}-du{3}-do{4:.3f}-rs{5:.3f}'.format(
        timestamp, 
        history.history['val_acc'][-1],
        history.history['val_loss'][-1],
        242,
        0.43,
        0.0001
    )).replace('.', ',')
    # Save CM
    cm.save_html(os.path.join(OUTPUT_DIR, '{}_confusion_matrix'.format(model_string)))
    # Save model
    save_model(
        model,
        os.path.join(OUTPUT_DIR, model_string + '.h5')
    )
    
    # Plot the training loss and accuracy
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, N_EPOCHS), history.history['loss'], label='train_loss')
    plt.plot(np.arange(0, N_EPOCHS), history.history['val_loss'], label='val_loss')
    plt.plot(np.arange(0, N_EPOCHS), history.history['acc'], label='train_acc')
    plt.plot(np.arange(0, N_EPOCHS), history.history['val_acc'], label='val_acc')
    plt.title('Training Loss and Accuracy')
    plt.xlabel('Epoch #')
    plt.ylabel('Loss/Accuracy')
    plt.legend()
    plt.savefig(os.path.join(OUTPUT_DIR, model_string+'.png'))
    
    try:
        # Report to SigOpt
        sigopt_connection.experiments(sigopt_experiment.id).observations().create(
            suggestion=suggestion.id,
            value=1.0/(history.history['val_loss'][-1])
        )
        # Update the experiment object
        sigopt_experiment = sigopt_connection.experiments(sigopt_experiment.id).fetch()
    except:
        pass
    
    nLoops -= 1

[INFO] ImageNet weights w/ Non-Standard Input Dimensions, redefining MobileNetV2 head...


[INFO] training network...


ValueError: Error when checking input: expected mobilenetv2_1.40_224_input to have shape (224, 224, 1) but got array with shape (224, 224, 3)

In [ ]:
# Fetch the best configuration and explore your experiment
best_assignments = sigopt_connection.experiments(sigopt_experiment.id).best_assignments().fetch().data[0].assignments
print("Best Assignments: " + best_assignments)
print("Explore your experiment: https://app.sigopt.com/experiment/" + sigopt_experiment.id + "/analysis")

## Here the most common ways to prevent overfitting in neural networks:

* Get more training data.
* Reduce the capacity of the network.
* Add weight regularization.
* Add dropout.